In [1]:
import importlib
import json
import pandas as pd
import numpy as np
import time
import warnings
import importlib
from sklearn.datasets import fetch_openml 
from xgboost import XGBClassifier, XGBRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.svm import SVC, SVR
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import PowerTransformer, OneHotEncoder, LabelEncoder
import os
warnings.filterwarnings('ignore')

# Ostateczna ocena konfiguracji i zapis do pliku

Przeprowadzimy teraz screening 50 konfiguracji na różnych zbioroach danych zOpenML (w tym niezbalansowanych), ograniczonych do 5000 próbek w celu szybkiej weryfikacji zdolnosci generalizacji oraz potencjału do roziwązywania wielu problemów.

In [2]:
datasets = {
    1510: "WBC (Breast Cancer)",    
    1462: "Banknote Auth",          
    37:   "Diabetes (Pima)",
    1461: "Bank Marketing",         
    1049: "PC4 (Software Defect)",  
    1464: "Blood Transfusion", 
    1485: "Madelon",                
    1479: "Hill-Valley",            
    40994: "Climate Crashes",
    31:   "German Credit",          
    44:   "Spambase",               
    1494: "QSAR Biodeg", 
}

In [3]:
def get_class(path):
    """Dynamically load a class from a configuration string."""
    module_path, class_name = path.rsplit('.', 1)
    module = importlib.import_module(module_path)
    return getattr(module, class_name)

def load_models_from_json(path):
    """Load model configurations from a JSON file."""
    with open(path, 'r') as f:
        try:
            models = json.load(f)
            print(f"Loaded {len(models)} models from file '{path}'")
            return models
        except json.JSONDecodeError as e:
            print(e)
            return []
        
# zdefiniujemy pipeline który użyjemy prawdopodobnie w systemie miniautoml
def build_pipeline(model, X):
    numeric_features = X.select_dtypes(include=['int64', 'float64']).columns
    categorical_features = X.select_dtypes(include=['object', 'category', 'bool']).columns
    
    numeric_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', PowerTransformer(method='yeo-johnson')) 
    ])
    
    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
        ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
    ])
    
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, numeric_features),
            ('cat', categorical_transformer, categorical_features)
        ],
        verbose_feature_names_out=False
    )
    
    return Pipeline(steps=[('preprocessor', preprocessor), ('classifier', model)])

# załadujemy dane z openml o danych z datasets; nie chcemy pełnych zbiorów tylko próbki do 5000 rekordów aby zorientować się w działaniu modeli

def load_openml(datesets_id, name):
    try:
        X,y = fetch_openml(data_id=datesets_id, as_frame=True, return_X_y=True, parser='auto')
        if len(X) > 5000:
            X = X.sample(5000, random_state=42)
            y = y.loc[X.index]
        le = LabelEncoder()
        y = le.fit_transform(y)
        if len(np.unique(y))!=2:
            print(f"Dataset {name} (ID: {datesets_id}) is not binary classification, skipping.")
            return None, None
        print(f"Loaded dataset {name} (ID: {datesets_id}) with {X.shape[0]} samples and {X.shape[1]} features.")
        return X, y
    except Exception as e:
        print(e)
        return None, None   

In [4]:
def run(json_path):
    models_config = load_models_from_json(json_path)
    if not models_config:
        print("No models to evaluate.")
        return
    results = []
    for d_id, d_name in datasets.items():
        print(f"\n--- RUNDA: {d_name} ---")
        X, y = load_openml(d_id, d_name)
        
        if X is None: continue
        
        for config in models_config:
            model_name = config['name']
            
            try:
                ModelClass = get_class(config['class'])
                clf = ModelClass(**config['params'])
                full_pipeline = build_pipeline(clf, X)
                
                start_time = time.time()
                scores = cross_val_score(full_pipeline, X, y, cv=3, scoring='roc_auc', error_score='raise')
                duration = time.time() - start_time
                
                mean_score = np.mean(scores)
                
                print(f"  > {model_name:<20} | AUC: {mean_score:.4f} | {duration:.2f}s")
                
                results.append({
                    'Dataset': d_name,
                    'Difficulty': 'Hard' if d_id in [1485, 1479, 40994] else 'Easy/Std',
                    'Model': model_name,
                    'Mean_AUC': mean_score,
                    'Time_Sec': duration
                })
                
            except Exception as e:
                print(f"  ! Error with model {model_name}")
    if not results: 
        print("Brak wyników.")
        return

    df_res = pd.DataFrame(results)
    
    print("\n" + "="*80)
    print(f"{' SUMMARY ':^80}")
    print("="*80)
    
    pivot = df_res.pivot_table(index='Model', columns='Dataset', values='Mean_AUC')
    
    pivot['Global_Avg'] = pivot.mean(axis=1)
    pivot = pivot.sort_values('Global_Avg', ascending=False)
    
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', 1000)
    pd.set_option('display.precision', 4)
    print(pivot)

    filename = 'results.csv'
    pivot.to_csv(filename)
    print(f"\nSaved in: {filename}")

In [5]:
run('models_for_tests.json')

Loaded 47 models from file 'models_for_tests.json'

--- RUNDA: WBC (Breast Cancer) ---
Loaded dataset WBC (Breast Cancer) (ID: 1510) with 569 samples and 30 features.
  > log_reg1             | AUC: 0.9960 | 0.25s
  > log_reg2             | AUC: 0.9959 | 6.81s
  > log_reg3             | AUC: 0.9949 | 0.58s
  > log_reg4             | AUC: 0.9938 | 0.17s
  > log_reg5             | AUC: 0.9925 | 0.13s
  > lda1                 | AUC: 0.9925 | 0.14s
  > lda2                 | AUC: 0.9900 | 0.14s
  > ridge_clf            | AUC: 0.9915 | 0.13s
  > decision_tree1       | AUC: 0.9104 | 0.14s
  > decision_tree2       | AUC: 0.8677 | 0.13s
  > decision_tree3       | AUC: 0.9510 | 0.14s
  > decision_tree4       | AUC: 0.9301 | 0.15s
  > decision_tree5       | AUC: 0.9180 | 0.22s
  > random_forest1       | AUC: 0.9903 | 0.52s
  > random_forest2       | AUC: 0.9906 | 1.23s
  > random_forest3       | AUC: 0.9876 | 1.82s
  > random_forest4       | AUC: 0.9915 | 0.87s
  > extra_trees1         | AUC: 0.

Zredukowałyśmy pulę z 50 do 30 modeli, aby odsiać konfiguracje nieefektywne, które mogłyby zaszumić wynik końcowego ensamblu, oraz aby zoptymalizować czas obliczeń, skupiając zasoby tylko na najbardziej obiecujących algorytmach.
Strategia selekcji:
 1. Scoring: Średnia AUC skorygowana o odchylenie standardowe (kara za niestabilność).
    Formula: Score = Mean_AUC - (PENALTY_FACTOR * Volatility).
 2. Etap 1 (Diversity): Wybór po jednym najlepszym reprezentancie z każdej rodziny 
    (Linear, Geometry/Bayes, RandomForest, Boosting).
 3. Etap 2 (Performance): Dopełnienie listy do 30 modeli na podstawie najwyższego Score.

Kolejność konfiguracji w pliku wyjściowym została zoptymalizowana pod kątem ograniczeń czasowych. Na szczycie listy priorytetów umieszczono liderów poszczególnych rodzin algorytmicznych. Taka strategia gwarantuje, że w przypadku konieczności przedwczesnego przerwania treningu, system będzie dysponował zróżnicowanym i stabilnym zestawem modeli bazowych, niezbędnym do skonstruowania skutecznego Ensemblu, unikając sytuacji, w której przetestowane zostałyby tylko modele jednego typu.

In [6]:
def detect_family(model_class):
    cls = model_class.lower()
    if 'linear' in cls or 'discriminant' in cls or 'ridge' in cls: 
        return 'Linear'
    if 'svm' in cls or 'neighbor' in cls or 'bayes' in cls: 
        return 'Geometry/Bayes'
    if 'randomforest' in cls or 'extratrees' in cls or 'decisiontree' in cls: 
        return 'RandomForest'
    if 'boost' in cls or 'lgbm' in cls: 
        return 'Boosting'
    return 'Other'

In [7]:
def run_optim(input_csv, input_models, output_json, penalty=0.15, size=30):
    if not os.path.exists(input_csv) or not os.path.exists(input_models):
        print(f" Missing {input_csv} or {input_models}")
        return
    df = pd.read_csv(input_csv)
    with open(input_models, 'r') as f:
        full_configs = json.load(f)
    
    config_map = {}
    for item in full_configs:
        if 'name' in item:
            item['family'] = detect_family(item.get('class', ''))
            config_map[item['name']] = item
    
    print(f"Len of scores{len(df)}, len of configs {len(config_map)}")

    cols = [c for c in df.columns if c not in ['Model', 'Global_Avg', 'Difficulty']]
    df['Real_Avg'] = df[cols].mean(axis=1)
    df['Volatility'] = df[cols].std(axis=1)
    df['Final_Score'] = df['Real_Avg'] - (penalty * df['Volatility'])
    df['Family'] = df['Model'].map(lambda x: config_map.get(x, {}).get('family', 'Unknown'))

    df = df.sort_values('Final_Score', ascending=False)
    final_models = []
    selected_names = set()

    families = ['Linear', 'Geometry/Bayes', 'RandomForest', 'Boosting']

    for fam in families:
        candidates = df[df['Family'] == fam]
        if not candidates.empty:
            winner = candidates.iloc[0]
            name = winner['Model']
            if name not in selected_names:
                final_models.append(name)
                selected_names.add(name)

    for _, row in df.iterrows():
        if len(final_models) >= size:
            break
        name = row['Model']
        if name not in selected_names:
            final_models.append(name)
            selected_names.add(name)
    output_list = []
    for name in final_models:
        if name in config_map:
            cfg = config_map[name].copy()
            if 'family' in cfg: del cfg['family']
            output_list.append(cfg)

    with open(output_json, 'w') as f:
        json.dump(output_list, f, indent=2)
    print(f"Saved {len(output_list)} selected models to {output_json}")

In [8]:
output_json = 'models.json'
input_json = 'models_for_tests.json'
input_csv = 'results.csv'

run_optim(input_csv, input_json, output_json, penalty=0.15, size=30)

Len of scores40, len of configs 47
Saved 30 selected models to models.json


W ostatecznym systemie decydujemy się na wybór:
* Preprocessing Danych:
    - Dla zmiennych numerycznych stosujemy imputację medianą (SimpleImputer) oraz zaawansowaną transformację Yeo-Johnsona (PowerTransformer), która stabilizuje wariancję i normalizuje rozkłady cech.
    - Dla zmiennych kategorycznych używamy imputacji stałą wartością ('missing') oraz kodowania One-Hot Encoding z obsługą nieznanych kategorii.
* Strategia Selekcji:
    - Wybór najlepszego algorytmu następuje na podstawie rankingu Balanced Accuracy, wyliczanego na wydzielonym zbiorze walidacyjnym (20% danych, podział stratyfikowany).
    - Dla każdego modelu system automatycznie optymalizuje próg decyzyjny (threshold) w zakresie od 0.2 do 0.8, co jest kluczowe przy niezbalansowanych klasach.
    - Proces selekcji jest ograniczony parametrami wydajnościowymi: maksymalnym czasem przeszukiwania (total_time_limit [default = 20 minut]) oraz limitem wierszy (max_rows_limit [default = 20000]) dla fazy turniejowej. total_time_limit nie jest w całości przeznaczony na fazę selekcji. Rezerwowane jest około 1/4 dostępnego czasu na trenowanie finalnego modelu na pełnym zbiorze danych
* Ensembling:
    - System buduje potencjalny model zespołowy VotingClassifier w trybie Soft Voting (uśrednianie prawdopodobieństw).
    - Do głosowania wybierane jest TOP K (parametr top_k, domyślnie 5) najlepszych modeli z turnieju. Algorytm priorytetyzuje różnorodność, starając się w pierwszej kolejności dobrać liderów z różnych rodzin (np. Liniowe, Drzewa, Boosting), aby zredukować korelację błędów.
    - Model zespołowy zostaje wybrany jako finalny tylko wtedy, gdy jego wynik walidacyjny przebija wynik najlepszego pojedynczego modelu (Single Model).
* Finalny trening: 
    - Zwycięska konfiguracja jest trenowana ponownie na pełnym zbiorze danych (z zachowaniem parametru random_state dla powtarzalności).

System MiniAutoML znajduje się w pliku automl.py.

Konfiguracje przekazywane do init są w pliku models.json.

Dodatkowo zdefiniowany został moduł run_function.py. Zawiera on funkcję sterującą, która przyjmuje ścieżkę do katalogu z danymi (X.csv, y.csv) i automatyzuje proces eksperymentalny.  
Przed uruchomieniem właściwego treningu, funkcja dokonuje podziału danych na zbiór treningowy (przekazywany do systemu AutoML) oraz zbiór testowy (ukryty przed systemem, służący do ostatecznej weryfikacji).  
Skrypt dostarcza również kluczowych statystyk, takich jak wymiary zbioru (liczba wierszy i cech) oraz balans klas. Po zakończeniu uczenia prezentuje wyniki wewnętrznego rankingu modeli (Leaderboard), a następnie ewaluuje zwycięski model na wydzielonym zbiorze testowym, raportując metryki skuteczności oraz automatycznie dobrany próg decyzyjny (threshold).